# Construcción de un DataFrame Completo con SWAPI
## Ejercicio - API REST y Pandas
Este notebook muestra cómo se obtiene información detallada de los personajes de la saga Star Wars usando la API de SWAPI, y cómo se procesa para construir un DataFrame en Pandas.


## Descripción del proceso

El objetivo principal es construir un **DataFrame** que contenga información sobre los personajes de Star Wars, utilizando la API de SWAPI.

### Pasos:

1. **Obtención de datos de personajes:**
   - Se utiliza la API de SWAPI para obtener información paginada sobre los personajes.

2. **Procesamiento de datos relacionados:**
   - Los campos como `homeworld`, `species`, y `films` contienen URLs que requieren peticiones adicionales para obtener sus valores descriptivos (nombres de planetas, especies, títulos de películas).

3. **Construcción del DataFrame:**
   - Se procesan los datos obtenidos y se almacenan en un DataFrame con las siguientes columnas:
     - `id`, `name`, `height`, `mass`, `hair_color`, `skin_color`, `eye_color`, `birth_year`, `gender`, `homeworld`, `species`, y `films`.

4. **Manejo de errores y datos faltantes:**
   - Se implementan controles para evitar que errores en la API o datos faltantes interrumpan el flujo del programa.
   - Se asignan valores predeterminados como `"Unknown"` o listas vacías para campos no disponibles.

5. **Exportación del DataFrame:**
   - El DataFrame resultante se guarda como un archivo `.csv` para análisis posteriores.


In [1]:
import requests
import pandas as pd
import time

# URL base de la API
BASE_URL = "https://swapi.dev/api/people/"

# Lista para almacenar los datos procesados
all_characters = []

# Función para obtener datos de todas las páginas
def fetch_all_characters(base_url):
    url = base_url
    while url:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Verifica si hay errores en la respuesta
            data = response.json()
            all_characters.extend(data['results'])  # Agrega los resultados actuales
            url = data['next']  # Obtiene la siguiente página
            time.sleep(0.5)  # Espera para evitar sobrecargar la API
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener los datos: {e}")
            break

# Llamar a la función para obtener todos los personajes
fetch_all_characters(BASE_URL)

In [2]:
def get_name_from_url(url, field="name"):
    if not url:
        return None
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get(field)  # Devuelve el valor del campo solicitado
    except requests.exceptions.RequestException:
        return None

# Función para obtener una lista de nombres desde una lista de URLs
def get_names_from_urls(urls, field="title"):
    names = []
    for url in urls:
        name = get_name_from_url(url, field)
        if name:
            names.append(name)
        time.sleep(0.5)  # Controla el número de solicitudes
    return names

In [3]:
# Procesar los datos para crear el DataFrame
processed_data = []
for i, character in enumerate(all_characters, start=1):
    # Manejar errores y campos faltantes
    homeworld_name = get_name_from_url(character.get("homeworld"), field="name")
    species_name = get_name_from_url(character["species"][0], field="name") if character["species"] else "Human"
    films_titles = get_names_from_urls(character.get("films", []), field="title")
    
    # Agregar los datos procesados a la lista
    processed_data.append({
        "id": i,
        "name": character.get("name", "Unknown"),  # Valor predeterminado si falta
        "height": character.get("height", "Unknown"),
        "mass": character.get("mass", "Unknown"),
        "hair_color": character.get("hair_color", "Unknown"),
        "skin_color": character.get("skin_color", "Unknown"),
        "eye_color": character.get("eye_color", "Unknown"),
        "birth_year": character.get("birth_year", "Unknown"),
        "gender": character.get("gender", "Unknown"),
        "homeworld": homeworld_name if homeworld_name else "Unknown",
        "species": species_name,
        "films": films_titles if films_titles else []
    })

In [4]:
# Crear el DataFrame
df = pd.DataFrame(processed_data)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Guardar el DataFrame como un archivo CSV
df.to_csv("star_wars_characters_detailed.csv", index=False)

   id            name height mass hair_color   skin_color eye_color  \
0   1  Luke Skywalker    172   77      blond         fair      blue   
1   2           C-3PO    167   75        n/a         gold    yellow   
2   3           R2-D2     96   32        n/a  white, blue       red   
3   4     Darth Vader    202  136       none        white    yellow   
4   5     Leia Organa    150   49      brown        light     brown   

  birth_year  gender homeworld species  \
0      19BBY    male  Tatooine   Human   
1     112BBY     n/a  Tatooine   Droid   
2      33BBY     n/a     Naboo   Droid   
3    41.9BBY    male  Tatooine   Human   
4      19BBY  female  Alderaan   Human   

                                               films  
0  [A New Hope, The Empire Strikes Back, Return o...  
1  [A New Hope, The Empire Strikes Back, Return o...  
2  [A New Hope, The Empire Strikes Back, Return o...  
3  [A New Hope, The Empire Strikes Back, Return o...  
4  [A New Hope, The Empire Strikes Back, Retu